之前我们已经完成了《Python数据分析基础》这本书的学习，之后会通过两篇笔记学习关联分析。
这篇笔记是关于关联分析和对应的一个经典数据挖掘算法的。
关联分析（Association Analysis）主要用于发现隐藏在大型数据集中的有意义的联系，它起源于商品销售领域，“啤酒与尿布”的故事体现的就是数据分析/挖掘领域非常经典的推荐方式，而现在各类互联网公司的推荐系统都有关联分析的影子。



### 目录
- 基本概念
- 几种关联分析算法
- Apriori的实现与应用


## 基本概念

**关联规则**一般表示为：“面包=>牛奶”，其中面包是规则的前项，牛奶为后项，关联规则是有方向性的，例如“面包=>牛奶”和“牛奶=>面包”是两个不同的关联规则；项集N、M同时发生的概率称为关联规则的**支持度**（support）；支持度计数：一个项集出现在几个事务当中，它的支持度计数就是几。例如{Diaper, Beer}出现在事务002、003和004中，所以它的支持度计数是3。 项集N发生，则项集M发生的概率称为关联规则的**置信度**（confidence），计算方法为：对于规则{Diaper}→{Beer}，{Diaper, Beer}的支持度计数除于{Diaper}的支持度计数。**频繁项集**：支持度大于或等于某个阈值的项集就叫做频繁项集。例如阈值设为50%时，因为{Diaper, Beer}的支持度是75%，所以它是频繁项集。 


## 几种关联分析算法

- Apriori：基于频繁项集原理，其核心思想是通过连接产生候选项及其支持度然后通过剪枝生成频繁项集；
- FP-growth：针对Apriori的固有多次扫描事务数据集的缺陷，提出的不产生候选频繁项集的方法。
- Eclat算法：一种深度优先算法，采用垂直数据表示方式，在**概念格理论**的基础上利用基于前缀的等价关系讲搜索空间划分为较小的子空间。
- 灰色关联算法：分析和确定各元素之间的影响程度或若干个子序列对母序列的贡献度而进行的一种分析方法。

本篇笔记会用代码详细说明Apriori算法，下一篇笔记会讲用于优化Apriori的FP-growth算法。


## Apriori

apriori在拉丁语中意思是“来自以前”，Apriori算法就是基于一个比较重要的先验知识：
如果一个项集是频繁项集，则它的子项集也都是频繁项集。如果一个项集是非频繁项集，则它的超集也一定是非频繁项集。

例如{Diaper, Beer}是频繁项集，则{Diaper}、{Beer}也都是频繁项集。 
例如{Egg}是非频繁项集，则{Cola, Egg}也是非频繁项集。）

下面看一下Apriori算法的实现过程：

In [11]:
def createC1(dataSet): #为数据集构建所有候选项集的集合
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return map(frozenset, C1)#对C1的每个项构建一个不变的集合

def scanD(D, Ck, minSupport):
    ssCnt = {}
    numIt=0
    for tid in D:
        numIt +=1
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    #numItems = float(len(list(D)))   #D:map
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numIt #计算所有项集的支持度
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

def aprioriGen(Lk, k): #创建Ck
    #传入参数：频繁项集列表Lk、项集元素个数k
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #前k-2个项相同时，合并集合
                retList.append(Lk[i] | Lk[j]) 
    return retList

def apriori(dataSet, minSupport = 0.5): #从C1得到L1
    C1 = createC1(dataSet)
    D = map(set, dataSet)
    #print(D)
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#扫描数据集，从CK得到Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData




data_set=[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
L,suptDict=apriori(data_set,0.1)   #调用
print(L)

[[frozenset({4}), frozenset({3}), frozenset({1})], []]


**关联规则的生成**：

In [13]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #用作返回值
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #计算置信度
        if conf >= minConf: 
            print( freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

#生成候选规则集合
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #进行进一步合并
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #创建Hm=1条新候选规则
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)
            
#调用前面的两个函数            
def generateRules(L, suptDataDict, minConf=0.7):  #supportData是字典格式
    #传入参数：频繁项集列表、包含那些频繁项集支持数据的字典、最小可信度阈值
    bigRuleList = []
    for i in range(1, len(L)):#只获取两个或更多元素的集合
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, suptDataDict, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1,suptDataDict,bigRuleList, minConf)
    return bigRuleList   


关联规则生成的代码包含3个函数，其中generateRules是主函数，调用其他的两个函数，rulesFromConseq()和calcConf()分别用于生成候选规则集合以及对规则进行评估，generateRules()的参数有频繁项集列表L、包含那些频繁项集支持数据的字典suptDataDict、最小可信度阈值minConf。

In [18]:
out_rules=generateRules(L, suptDict,0.8) #调用
out_rules 

[]

Apriori算法基于先验知识来挖掘数据集中隐含的关联规则，效果挺好，它通过Apriori原理来减少在数据集上检查集合的数目，但是每次增加频繁项集的大小，Apriori算法就需要重新扫描整个数据集，在大数据环境下，这是比较低效的。因此基于Apriori原理产生了一个更高效的算法，该算法会使用到上一章笔记中讲到过的树结构。FP-growth算法只需要对数据集进行两次遍历，速度会快很多。


#### 参考资料：

- [《机器学习实战》](https://book.douban.com/subject/24703171/) .Peter Harrington.人民邮电出版社.
- [《数据挖掘导论》](https://book.douban.com/subject/5377669/).Pang-Ning Tan 等.人民邮电出版社
- [《Python数据分析与挖掘实战》](https://book.douban.com/subject/26677686/). 张良均 等.机械工业出版社